In [2]:
import pandas as pd
import numpy as np
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings
import faiss
import os
import math
import json
import time
import requests
import numpy as np

In [8]:
RERANKER_URL = "https://ai-for-finance-hack.up.railway.app/rerank"
RERANKER_MODEL = "deepinfra/Qwen/Qwen3-Reranker-4B"
EMBEDDER_API_KEY = os.getenv("EMBEDDER_API_KEY")
LLM_API_KEY =  os.getenv("LLM_API_KEY")


In [4]:
TOP_K = 60   
TOP_R = 6    

In [5]:
loader = CSVLoader(file_path = "train_data.csv")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100, add_start_index = True)
all_splits = text_splitter.split_documents(docs)

In [6]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=EMBEDDER_API_KEY,
    base_url="https://ai-for-finance-hack.up.railway.app/"
)

In [7]:
chunk_texts = [d.page_content for d in all_splits]
chunk_vecs_list = embeddings.embed_documents(chunk_texts) 
chunk_vecs = np.array(chunk_vecs_list, dtype=np.float32)  

KeyboardInterrupt: 

In [ ]:
norms = np.linalg.norm(chunk_vecs, axis=1, keepdims=True)
norms[norms == 0] = 1.0
chunk_vecs = chunk_vecs / norms

In [ ]:
def embedding_query(query: str):
    qv = embeddings.embed_query(query)
    qv = np.array(qv, dtype=np.float32)
    qnorm = np.linalg.norm(qv)
    if qnorm == 0:
        qnorm = 1.0
    qv = qv / qnorm
    return qv


In [ ]:
def retrieve_top_k_by_cosine(query: str, top_k: int = TOP_K):
    qv = embedding_query(query)
    sims = chunk_vecs @ qv  
    idx = np.argsort(-sims)[:top_k]
    return [(int(i), float(sims[i])) for i in idx]

In [ ]:
def call_reranker(query: str, documents: list, key: str = EMBEDDER_API_KEY, model: str = RERANKER_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {key}"
    }
    payload = {
        "model": model,
        "query": query,
        "documents": documents
    }
    resp = requests.post(RERANKER_URL, headers=headers, json=payload)
    resp.raise_for_status()
    return resp.json()


In [ ]:
# Функция для генерации ответа по заданному вопросу, вы можете изменять ее в процессе работы, однако
# просим оставить структуру обращения, т.к. при запуске на сервере, потребуется корректно указанный путь 
# для формирования ответов. Также не вставляйте ключ вручную, поскольку при запуске ключ подтянется автоматически
def answer_generation(question):
	candidates = retrieve_top_k_by_cosine(question, top_k=TOP_K)
	cand_idxs = [i for i, s in candidates]
	cand_texts = [chunk_texts[i] for i in cand_idxs]
    
	rr_json = call_reranker(query=question, documents=cand_texts, key=EMBEDDER_API_KEY)
	results = rr_json.get("results", [])
     
	rr_scores = [r.get("relevance_score", 0.0) for r in results]
	pairs = list(zip(cand_idxs, rr_scores, cand_texts))
	pairs_sorted = sorted(pairs, key=lambda x: -x[1])
     
	top_pairs = pairs_sorted[:TOP_R]
	contexts = []
	provenance = []
	for idx, score, txt in top_pairs:
		contexts.append(f"[score={score:.4f}] {txt}")
		provenance.append({"chunk_index": idx}) 
    
	context = "\n\n---\n\n".join(contexts)

    # Подключаемся к модели
	client = OpenAI(
        # Базовый url - сохранять без изменения
        base_url="https://ai-for-finance-hack.up.railway.app/",
        # Указываем наш ключ, полученный ранее
        api_key=LLM_API_KEY,
    )
    # Формируем запрос к клиенту
	response = client.chat.completions.create(
        # Выбираем любую допступную модель из предоставленного списка
        model="openrouter/mistralai/mistral-small-3.2-24b-instruct",
        # Формируем сообщение
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"""
						Ты — ассистент, отвечающий на вопросы по данным.
						Используй только информацию из контекста ниже.

						Контекст:
						{context}

						Вопрос:
						{question}

						Если ответа нет в контексте — так и скажи: "Ответа в данных нет."
						Ответ:
						"""
                    }
                ]
            }
        ]
    )
    # Формируем ответ на запрос и возвращаем его в результате работы функции
	return response.choices[0].message.content


# Блок кода для запуска. Пожалуйста оставляйте его в самом низу вашего скрипта,
# при необходимости добавить код - опишите функции выше и вставьте их вызов в блок после if
# в том порядке, в котором они нужны для запуска решения, пути к файлам оставьте неизменными.
if __name__ == "__main__":    
    # Считываем список вопросов
    questions = pd.read_csv('./questions.csv')
    # Выделяем список вопросов
    questions_list = questions['Вопрос'].tolist()
    # Создаем список для хранения ответов
    answer_list = []
    # Проходимся по списку вопросов
    for current_question in tqdm(questions_list, desc="Генерация ответов"):
        # Отправляем запрос на генерацию ответа
        answer = answer_generation(question=current_question)
        # Добавляем ответ в список
        answer_list.append(answer)
    # Добавляем в данные список ответов
    questions['Ответы на вопрос'] = answer_list
    # Сохраняем submission
    questions.to_csv('submission.csv', index=False)

Генерация ответов:   1%|          | 6/500 [02:14<3:04:20, 22.39s/it]


KeyboardInterrupt: 